## Imports libraries

In [2]:
import torch
import torch.nn as nn

## Discriminator

In [3]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels,stride):
        super(CNNBlock,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, 4,stride,bias=False,padding_mode="reflect"
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )
    def forward(self, x):
        return self.conv(x)

## Generatot

## Config

## Utils

## Dataset